<a href="https://colab.research.google.com/github/netmatze/mlmatze/blob/main/cnn_flowers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### import standard libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

### import tensorflow and keras libraries

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from keras.preprocessing import image

### define the resolution for the images and the name of the dataset

In [ ]:
# dataset_name = 'tf_flowers'
# IMAGE_RESOLUTION = 224 #28
# CHANNELS = 3
# IMAGE_RESOLUTION = 28
# dataset_name = 'fashion_mnist'
# CHANNELS = 1
# IMAGE_RESOLUTION = 224 #28
# dataset_name = 'oxford_flowers102'
# CHANNELS = 3
# dataset_name = 'malaria'
# IMAGE_RESOLUTION = 128 #28
# CHANNELS = 3
dataset_name = 'plant_village'
IMAGE_RESOLUTION = 224 #28
CHANNELS = 3

### functions to preporocess the datasets

In [ ]:
def preprocess(item):
    image = item['image']
    label = item['label']
    return image, label

def preprocess_ds(item):
    image = item[0]
    label = item[1]
    return image, label

def preprocess_data_ds(image_data, label_data):
    image = image_data
    label = label_data
    # Resize the image to your desired size
    image = tf.image.resize(image, (IMAGE_RESOLUTION, IMAGE_RESOLUTION))
    # Normalize the image
    image = image / 255.0
    return image, label

### transform image to specific resolution

In [ ]:
def format_image(image, label, image_res = IMAGE_RESOLUTION):
    image = tf.image.resize(image, (image_res, image_res))/255.0
    return image, label

### functions to display the datasets

In [ ]:
def display_train_generator(train_generator, use_cmap = True, columns : int = 5, rows : int = 5):
  fig = plt.figure(figsize=(16, 16))
  for i in range(1, columns*rows + 1):
      x,y = train_generator.next()
      image = x[i]
      label = y[i]
      label_name = info.features['label'].names[label]
      fig.add_subplot(columns, rows, i)
      # plt.imshow(image, cmap = plt.get_cmap("plasma"))
      if use_cmap:
          plt.imshow(image.numpy()[: ,: , 0].astype(np.float32), cmap = plt.get_cmap("plasma"))
      else:
          plt.imshow(image)
      plt.imshow(image[: ,: , 0].astype(np.float32)) #, cmap = plt.get_cmap("plasma")
      plt.title(label=label_name)
  plt.show()

def display_dataset(dataset_train, use_cmap = True, columns : int = 5, rows : int = 5):
    item_list = []
    counter = 0
    for foo in dataset_train:
        image, label = preprocess(foo)
        if counter > columns*rows+1:
          break
        counter += 1
        label_name = info.features['label'].names[label]
        item_dict = {"label": label, "image": image, "label_name": label_name}
        item_list.append(item_dict)

    fig = plt.figure(figsize=(16, 16))
    for i in range(1, columns*rows +1):
        item = item_list[i]
        label = item["label"]
        image = item['image']
        label_name = item["label_name"]
        fig.add_subplot(rows, columns, i)
        if use_cmap:
          plt.imshow(image.numpy()[: ,: , 0].astype(np.float32), cmap = plt.get_cmap("plasma"))
        else:
          plt.imshow(image)
        plt.title(label=label_name)
    plt.show

### functions to display the predictions of the images

In [ ]:
def process_dataset_with_model_ds(dataset_train, model, height : int = 32, width : int = 32, columns : int = 5, rows : int = 5):
    item_list = []
    counter = 0
    for foo in dataset_train:
        image, label = preprocess_ds(foo)
        if counter > columns*rows+1:
          break
        counter += 1
        label_name = info.features['label'].names[label]
        item_dict = {"label": label, "image": image, "label_name": label_name}
        item_list.append(item_dict)

    fig = plt.figure(figsize=(16, 16))
    for i in range(1, columns*rows +1):
        item = item_list[i]
        label = item["label"]
        image = item['image']
        label_name = item["label_name"]

        image_resize = tf.image.resize(image, (IMAGE_RESOLUTION, IMAGE_RESOLUTION)) / 255.0
        image_redim = tf.expand_dims(image_resize, axis=0) #.shape.as_list()
        prediction = model.predict(image_redim)
        b = tf.math.argmax(prediction[0])

        max_item = max(prediction[0])
        index_value = [index for index, item in enumerate(prediction[0]) if item == max_item]
        index_value_int = int(index_value[0])

        label_name = info.features['label'].names[label]
        prediction_label_name = info.features['label'].names[index_value_int]

        color = "blue" if label == index_value_int else "red"

        label_finished = label_name + ' ' + str(label.numpy()) + ' / ' + prediction_label_name + ' ' + str(index_value_int) + ' '

        fig.add_subplot(rows, columns, i)
        plt.imshow(image.numpy()[: ,: , 0].astype(np.float32), cmap = plt.get_cmap("plasma"))
        plt.title(label=label_finished, color=color)
    plt.show()

def process_dataset_with_model(dataset_train, model, height : int = 32, width : int = 32, columns : int = 5, rows : int = 5):
    item_list = []
    counter = 0
    for foo in dataset_train:
        image, label = preprocess(foo)
        if counter > columns*rows+1:
          break
        counter += 1
        label_name = info.features['label'].names[label]
        item_dict = {"label": label, "image": image, "label_name": label_name}
        item_list.append(item_dict)

    fig = plt.figure(figsize=(16, 16))
    for i in range(1, columns*rows +1):
        item = item_list[i]
        label = item["label"]
        image = item['image']
        label_name = item["label_name"]

        image_resize = tf.image.resize(image, (IMAGE_RESOLUTION, IMAGE_RESOLUTION)) / 255.0
        image_redim = tf.expand_dims(image_resize, axis=0) #.shape.as_list()
        prediction = model.predict(image_redim)
        b = tf.math.argmax(prediction[0])

        max_item = max(prediction[0])
        index_value = [index for index, item in enumerate(prediction[0]) if item == max_item]
        index_value_int = int(index_value[0])

        label_name = info.features['label'].names[label]
        prediction_label_name = info.features['label'].names[index_value_int]

        color = "blue" if label == index_value_int else "red"

        label_finished = label_name + ' ' + str(label.numpy()) + ' / ' + prediction_label_name + ' ' + str(index_value_int) + ' '

        fig.add_subplot(rows, columns, i)
        plt.imshow(image.numpy()[: ,: , 0].astype(np.float32), cmap = plt.get_cmap("plasma"))
        plt.title(label=label_finished, color=color)
    plt.show()

### display training history

In [ ]:
def display_training_history(history, epochs):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(epochs)

    plt.figure(figsize=(16, 10))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()

### train custom convolutional neuronal network

In [ ]:
dataset, info = tfds.load(name=dataset_name, with_info=True)
dataset_train = dataset['train']

display_dataset(dataset_train)

(train_ds, test_ds), dataset_info = tfds.load(dataset_name,
                                              with_info=True,
                                              as_supervised=True,
                                              split=["train[:80%]", "train[:20%]"])

train_ds_original = train_ds
test_ds_original = test_ds

number_of_classes = dataset_info.features['label'].num_classes
resolution_of_first_image = dataset_info.features['image'].shape
counter = 1
for foo in dataset_train:
    image, label = preprocess(foo)
    print('Resolution of the {} image: {}'.format(counter, image.shape))
    counter = counter + 1
    if counter > 5:
      break
num_training_examples = 0
num_validation_examples = 0

for _ in train_ds:
    num_training_examples +=1

for _ in test_ds:
    num_validation_examples +=1

print('Total Number of Classes: {}'.format(number_of_classes))
print('Size of images: {}'.format(IMAGE_RESOLUTION))
print('Total Number of Training Images: {}'.format(num_training_examples))
print('Total Number of Validation Images: {} \n'.format(num_validation_examples))

BATCH_SIZE = 32
train_ds = train_ds.shuffle(num_training_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
test_ds = test_ds.map(format_image).batch(BATCH_SIZE).prefetch(1)

FILTER1_SIZE = 32
FILTER2_SIZE = 64
FILTER_SHAPE = (3, 3)
FULLY_CONNECT_NUM = 128
POOL_SHAPE = (2, 2)
BATCH_SIZE = 32

activation_function = 'LeakyReLU'

cnn_model = tf.keras.Sequential([
    layers.Conv2D(FILTER1_SIZE, FILTER_SHAPE, input_shape=(IMAGE_RESOLUTION, IMAGE_RESOLUTION, CHANNELS), name='input'),
    layers.MaxPooling2D(POOL_SHAPE, name='first_pooling'),
    layers.Conv2D(FILTER2_SIZE, FILTER_SHAPE, activation=activation_function, name='second_convolution'),
    layers.MaxPooling2D(POOL_SHAPE, name='second_pooling'),
    layers.Conv2D(FILTER2_SIZE, FILTER_SHAPE, activation=activation_function, name='third_convolution'),
    layers.MaxPooling2D(POOL_SHAPE, name='third_pooling'),
    layers.Flatten(),
    layers.Dense(FULLY_CONNECT_NUM, activation=activation_function, name='dense_layer'),
    layers.Dense(number_of_classes, activation='softmax', name='output')
])

# opt = tf.keras.optimizers.Adam(learning_rate=0.001)
# optimizer = tf.keras.optimizers.AdamW(learning_rate=0.001)
# opt = tf.keras.optimizers.RMSprop()

optimizer = tf.keras.optimizers.AdamW(learning_rate=0.001)
loss = tf.keras.losses.SparseCategoricalCrossentropy()
metrics = ['accuracy']

cnn_model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics
)

cnn_model.summary()

learning_reduce = ReduceLROnPlateau(monitor='val_accuracy', factor=0.6, patience=5, verbose=1, mode='max', min_lr=5e-7)

early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)

EPOCHS = 20
history = cnn_model.fit(train_ds, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=test_ds) #, callbacks=[learning_reduce,early_stopping]) #train_ds

print("Evaluate on train data")
results = cnn_model.evaluate(train_ds, batch_size=32)
print("train loss, train acc:", results)

print("Evaluate on test data")
results = cnn_model.evaluate(test_ds, batch_size=32)
print("test loss, test acc:", results)

process_dataset_with_model_ds(train_ds_original, model=cnn_model, height=28, width=28)
process_dataset_with_model_ds(test_ds_original, model=cnn_model, height=28, width=28)
display_training_history(history, EPOCHS)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...: 0 url [00:00, ? url/s]

...

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

DownloadRetryLimitReachedError: ignored

### create convolutional neuronal network with data augmentation

In [ ]:
dataset, info = tfds.load(name=dataset_name, with_info=True)
dataset_train = dataset['train']

(train_ds, test_ds), dataset_info = tfds.load(dataset_name,
                                              with_info=True,
                                              as_supervised=True,
                                              split=["train[:80%]", "train[:20%]"])
train_ds_original = train_ds
test_ds_original = test_ds

number_of_classes = dataset_info.features['label'].num_classes
num_training_examples = 0
num_validation_examples = 0

for _ in train_ds_original:
    num_training_examples +=1

for _ in test_ds_original:
    num_validation_examples +=1

print('Total Number of Classes: {}'.format(number_of_classes))
print('Size of images: {}'.format(IMAGE_RESOLUTION))
print('Total Number of Training Images: {}'.format(num_training_examples))
print('Total Number of Validation Images: {} \n'.format(num_validation_examples))

train_ds = train_ds.map(preprocess_data_ds)
test_ds = test_ds.map(preprocess_data_ds)

data_generator = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

train_x_batches = train_ds.map(lambda x, y: x)
train_y_batches = train_ds.map(lambda x, y: y)

test_x_batches = test_ds.map(lambda x, y: x)
test_y_batches = test_ds.map(lambda x, y: y)

train_x_list = tf.constant(list(train_x_batches.as_numpy_iterator()))
train_y_list = tf.constant(list(train_y_batches.as_numpy_iterator()))

test_x_list = tf.constant(list(test_x_batches.as_numpy_iterator()))
test_y_list = tf.constant(list(test_y_batches.as_numpy_iterator()))

train_generator = data_generator.flow(train_x_list, train_y_list,batch_size=32)

print('Total Number of data generated Images: {} \n'.format(train_generator.n))

display_train_generator(train_generator=train_generator)

FILTER1_SIZE = 64
FILTER2_SIZE = 128
FILTER_SHAPE = (2, 2)
FULLY_CONNECT_NUM = 156
POOL_SHAPE = (2, 2)
BATCH_SIZE = 32

activation_function = 'LeakyReLU'

cnn_model = tf.keras.Sequential([
    layers.Conv2D(FILTER1_SIZE, FILTER_SHAPE, input_shape=(IMAGE_RESOLUTION, IMAGE_RESOLUTION, CHANNELS), name='input'),
    layers.MaxPooling2D(POOL_SHAPE, name='first_pooling'),
    layers.Conv2D(FILTER2_SIZE, FILTER_SHAPE, activation=activation_function, name='second_convolution'),
    layers.MaxPooling2D(POOL_SHAPE, name='second_pooling'),
    layers.Conv2D(FILTER2_SIZE, FILTER_SHAPE, activation=activation_function, name='third_convolution'),
    layers.MaxPooling2D(POOL_SHAPE, name='third_pooling'),
    layers.Flatten(),
    layers.Dense(FULLY_CONNECT_NUM, activation=activation_function, name='full_dense'),
    layers.Dense(number_of_classes, activation='softmax', name='output')
])

# opt = tf.keras.optimizers.Adam(learning_rate=0.001)
opt = tf.keras.optimizers.AdamW(learning_rate=0.001)

loss = tf.keras.losses.SparseCategoricalCrossentropy()

metrics = ['accuracy']

cnn_model.compile(
    optimizer=opt,
    loss=loss,
    metrics=metrics
)

cnn_model.summary()

learning_reduce = ReduceLROnPlateau(monitor='val_accuracy', factor=0.6, patience=5, verbose=1, mode='max', min_lr=5e-7)

early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)

EPOCHS = 5

history = cnn_model.fit(train_generator, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(test_x_list, test_y_list), callbacks=[learning_reduce, early_stopping]) #train_ds ,

print("Evaluate on train generator data")
results = cnn_model.evaluate(train_generator)
print("train generator loss, train generator acc:", results)

print("Evaluate on train data")
results = cnn_model.evaluate(train_x_list, train_y_list)
print("train loss, train acc:", results)

print("Evaluate on test data")
results = cnn_model.evaluate(test_x_list, test_y_list)
print("test loss, test acc:", results)

process_dataset_with_model_ds(train_ds_original, cnn_model)
process_dataset_with_model_ds(test_ds_original, cnn_model)
display_training_history(history, EPOCHS)

### rebuild the vgg16 convolutional model using data augmentation

In [ ]:
dataset, info = tfds.load(name=dataset_name, with_info=True)
dataset_train = dataset['train']

(train_ds, test_ds), dataset_info = tfds.load(dataset_name,
                                              with_info=True,
                                              as_supervised=True,
                                              split=["train[:80%]", "train[:20%]"])
train_ds_original = train_ds
test_ds_original = test_ds

number_of_classes = dataset_info.features['label'].num_classes
num_training_examples = 0
num_validation_examples = 0

for _ in train_ds_original:
    num_training_examples +=1

for _ in test_ds_original:
    num_validation_examples +=1

print('Total Number of Classes: {}'.format(number_of_classes))
print('Size of images: {}'.format(IMAGE_RESOLUTION))
print('Total Number of Training Images: {}'.format(num_training_examples))
print('Total Number of Validation Images: {} \n'.format(num_validation_examples))

train_ds = train_ds.map(preprocess_data_ds)
test_ds = test_ds.map(preprocess_data_ds)

data_generator = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

train_x_batches = train_ds.map(lambda x, y: x)
train_y_batches = train_ds.map(lambda x, y: y)

test_x_batches = test_ds.map(lambda x, y: x)
test_y_batches = test_ds.map(lambda x, y: y)

train_x_list = tf.constant(list(train_x_batches.as_numpy_iterator()))
train_y_list = tf.constant(list(train_y_batches.as_numpy_iterator()))

test_x_list = tf.constant(list(test_x_batches.as_numpy_iterator()))
test_y_list = tf.constant(list(test_y_batches.as_numpy_iterator()))

train_generator = data_generator.flow(train_x_list, train_y_list)

print('Total Number of data generated Images: {} \n'.format(train_generator.n))

display_train_generator(train_generator=train_generator)

cnn_model = Sequential()

BATCH_SIZE = 32
activation_function = 'LeakyReLU'

# Block 1
cnn_model.add(layers.Conv2D(64, (3, 3), activation=activation_function, padding='same', input_shape=(IMAGE_RESOLUTION, IMAGE_RESOLUTION, CHANNELS)))
cnn_model.add(layers.Conv2D(64, (3, 3), activation=activation_function, padding='same'))
cnn_model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

# Block 2
cnn_model.add(layers.Conv2D(128, (3, 3), activation=activation_function, padding='same'))
cnn_model.add(layers.Conv2D(128, (3, 3), activation=activation_function, padding='same'))
cnn_model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

# Block 3
# cnn_model.add(layers.Conv2D(256, (3, 3), activation=activation_function, padding='same'))
# cnn_model.add(layers.Conv2D(256, (3, 3), activation=activation_function, padding='same'))
# cnn_model.add(layers.Conv2D(256, (3, 3), activation=activation_function, padding='same'))
# cnn_model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

# # Block 4
# cnn_model.add(layers.Conv2D(512, (3, 3), activation=activation_function, padding='same'))
# cnn_model.add(layers.Conv2D(512, (3, 3), activation=activation_function, padding='same'))
# cnn_model.add(layers.Conv2D(512, (3, 3), activation=activation_function, padding='same'))
# cnn_model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

# # Block 5
# cnn_model.add(layers.Conv2D(512, (3, 3), activation=activation_function, padding='same'))
# cnn_model.add(layers.Conv2D(512, (3, 3), activation=activation_function, padding='same'))
# cnn_model.add(layers.Conv2D(512, (3, 3), activation=activation_function, padding='same'))
# cnn_model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

# Classification layers
cnn_model.add(layers.Flatten())
cnn_model.add(layers.Dense(156, activation=activation_function))
# cnn_model.add(layers.Dense(4096, activation=activation_function))
cnn_model.add(layers.Dense(number_of_classes, activation='softmax'))

# opt = tf.keras.optimizers.Adam(learning_rate=0.001)
opt = tf.keras.optimizers.AdamW(learning_rate=0.001)

loss = tf.keras.losses.SparseCategoricalCrossentropy()

metrics = ['accuracy']

cnn_model.compile(
    optimizer=opt,
    loss=loss,
    metrics=metrics
)

cnn_model.summary()

learning_reduce = ReduceLROnPlateau(monitor='val_accuracy', factor=0.6, patience=5, verbose=1, mode='max', min_lr=5e-7)

early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)

EPOCHS = 50

history = cnn_model.fit(train_generator, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(test_x_list, test_y_list), callbacks=[learning_reduce, early_stopping]) #train_ds ,

print("Evaluate on train generator data")
results = cnn_model.evaluate(train_generator)
print("train generator loss, train generator acc:", results)

print("Evaluate on train data")
results = cnn_model.evaluate(train_x_list, train_y_list)
print("train loss, train acc:", results)

print("Evaluate on test data")
results = cnn_model.evaluate(test_x_list, test_y_list)
print("test loss, test acc:", results)

process_dataset_with_model_ds(train_ds_original, cnn_model, height=224, width=224)
process_dataset_with_model_ds(test_ds_original, cnn_model, height=224, width=224)
display_training_history(history, EPOCHS)

### transfer learning - using feature extraction of pretrained MobileNetV2 model

In [ ]:
# def format_image(image, label, image_res = 224):
#     image = tf.image.resize(image, (image_res, image_res))/255.0
#     return image, label

dataset_name = 'tf_flowers'

datasets, info = tfds.load(name=dataset_name, with_info=True)
dataset_train = datasets['train']

display_dataset(dataset_train)

(train_ds, test_ds), dataset_info = tfds.load(dataset_name,
                                              with_info=True,
                                              as_supervised=True,
                                              split=["train[:80%]", "train[:20%]"])

number_of_classes = dataset_info.features['label'].num_classes
num_training_examples = 0
num_validation_examples = 0

for _ in train_ds:
    num_training_examples +=1

for _ in test_ds:
    num_validation_examples +=1

BATCH_SIZE = 32
train_ds = train_ds.shuffle(num_training_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
test_ds = test_ds.map(format_image).batch(BATCH_SIZE).prefetch(1)

opt = tf.keras.optimizers.Adam(learning_rate=1e-2)

algorithm = "MobileNetV2"

base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
            weights="imagenet", include_top=False, input_shape=(IMAGE_RESOLUTION, IMAGE_RESOLUTION, CHANNELS))
base_model.trainable = False
print('count layers of ' + algorithm + ' : ' + str(len(base_model.layers)))

base_model.summary()

model = tf.keras.Sequential()
model.add(base_model)
model.add(layers.GlobalMaxPool2D())
model.add(layers.Dense(number_of_classes, activation='softmax'))

opt = tf.keras.optimizers.Adam(learning_rate=1e-2)

loss = tf.keras.losses.SparseCategoricalCrossentropy()

metrics = ['accuracy']

model.compile(
    optimizer=opt,
    loss=loss,
    metrics=metrics
)

model.summary()

learning_reduce = ReduceLROnPlateau(monitor='val_accuracy', factor=0.6, patience=5, verbose=1, mode='max', min_lr=5e-7)

early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)

EPOCHS = 30

history = model.fit(train_ds, epochs=EPOCHS, validation_data=test_ds, callbacks=[learning_reduce,early_stopping])

print("Evaluate on train data")
results = model.evaluate(train_ds, batch_size=32)
print("train loss, train acc:", results)

print("Evaluate on test data")
results = model.evaluate(test_ds, batch_size=32)
print("test loss, test acc:", results)

process_dataset_with_model(dataset_train, model, height=224, width=224)

### transfer learning - using feature extraction of pretrained vgg16 model

In [ ]:
datasets, info = tfds.load(name=dataset_name, with_info=True)
dataset_train = datasets['train']

display_dataset(dataset_train)

(train_ds, test_ds), dataset_info = tfds.load(dataset_name,
                                              with_info=True,
                                              as_supervised=True,
                                              split=["train[:80%]", "train[:20%]"])

train_ds_original = train_ds
test_ds_original = test_ds

number_of_classes = dataset_info.features['label'].num_classes
num_training_examples = 0
num_validation_examples = 0

for _ in train_ds:
    num_training_examples +=1

for _ in test_ds:
    num_validation_examples +=1

BATCH_SIZE = 32
train_ds = train_ds.shuffle(num_training_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
test_ds = test_ds.map(format_image).batch(BATCH_SIZE).prefetch(1)

print(IMAGE_RESOLUTION)
base_model = tf.keras.applications.vgg16.VGG16(
            weights = 'imagenet', include_top=False, input_shape=(IMAGE_RESOLUTION, IMAGE_RESOLUTION, CHANNELS))
base_model.trainable = False
print('count layers of vgg16 : ' + str(len(base_model.layers)))

base_model.summary()

cnn_model = tf.keras.Sequential()
cnn_model.add(base_model)
cnn_model.add(layers.GlobalMaxPool2D())
cnn_model.add(layers.Dense(number_of_classes, activation='softmax'))

opt = tf.keras.optimizers.Adam(learning_rate=1e-2)

loss = tf.keras.losses.SparseCategoricalCrossentropy()

metrics = ['accuracy']

cnn_model.compile(
    optimizer=opt,
    loss=loss,
    metrics=metrics
)

cnn_model.summary()

learning_reduce = ReduceLROnPlateau(monitor='val_accuracy', factor=0.6, patience=5, verbose=1, mode='max', min_lr=5e-7)

early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)

EPOCHS = 20

history = cnn_model.fit(train_ds, epochs=EPOCHS, validation_data=test_ds, callbacks=[learning_reduce,early_stopping])

print("Evaluate on train data")
results = cnn_model.evaluate(train_ds, batch_size=32)
print("train loss, train acc:", results)

print("Evaluate on test data")
results = cnn_model.evaluate(test_ds, batch_size=32)
print("test loss, test acc:", results)

process_dataset_with_model_ds(train_ds_original, model=cnn_model)
process_dataset_with_model_ds(test_ds_original, model=cnn_model)
# display_training_history(history, EPOCHS)

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(16, 10))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
# plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
# plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

### transfer learning - using feature extraction of different pretrained models

In [ ]:
# VGG16/VGG19/MobleNetV2/EfficientNetV2B0/resnet_v2_50
def select_algorithm(algorithm = 'vgg16', input_resolution = 224, channels = 3):
    if algorithm == "vgg19":
        base_model = tf.keras.applications.vgg19.VGG19(
            weights = 'imagenet', include_top=False, input_shape=(input_resolution, input_resolution, channels))
    elif algorithm == "ResNet50":
        base_model = tf.keras.applications.resnet50.ResNet50(
            weights = 'imagenet', include_top=False, input_shape=(input_resolution, input_resolution, channels))
    elif algorithm == "MobileNetV2":
        base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
            weights="imagenet", include_top=False, input_shape=(input_resolution, input_resolution, channels))
    elif algorithm == "EfficientNetV2B0":
        base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(
            weights = 'imagenet', include_top=False, input_shape=(input_resolution, input_resolution, channels))
    else:
        base_model = tf.keras.applications.vgg16.VGG16(
            weights = 'imagenet', include_top=False, input_shape=(input_resolution, input_resolution, channels))
    base_model.trainable = False
    return base_model

In [ ]:
algorithm = 'EfficientNetV2B0'

base_model = select_algorithm(algorithm=algorithm, channels=CHANNELS)
print('count layers of ' + algorithm + ' : ' + str(len(base_model.layers)))

base_model.summary()

cnn_model = tf.keras.Sequential()
cnn_model.add(base_model)
cnn_model.add(layers.GlobalMaxPool2D())
cnn_model.add(layers.Dense(number_of_classes, activation='softmax'))

opt = tf.keras.optimizers.Adam(learning_rate=1e-2)

loss = tf.keras.losses.SparseCategoricalCrossentropy()

metrics = ['accuracy']

cnn_model.compile(
    optimizer=opt,
    loss=loss,
    metrics=metrics
)

cnn_model.summary()

learning_reduce = ReduceLROnPlateau(monitor='val_accuracy', factor=0.6, patience=5, verbose=1, mode='max', min_lr=5e-7)

early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)

EPOCHS = 20

history = cnn_model.fit(train_ds, epochs=EPOCHS, validation_data=test_ds, callbacks=[learning_reduce,early_stopping])

print("Evaluate on train data")
results = cnn_model.evaluate(train_ds, batch_size=32)
print("train loss, train acc:", results)

print("Evaluate on test data")
results = cnn_model.evaluate(test_ds, batch_size=32)
print("test loss, test acc:", results)

process_dataset_with_model_ds(train_ds_original, model=cnn_model)
process_dataset_with_model_ds(test_ds_original, model=cnn_model)
display_training_history(history, EPOCHS)

### transfer learning - using fine tuning training the pretrained vgg16 model

In [ ]:
base_model = tf.keras.applications.vgg16.VGG16(
  weights = 'imagenet', include_top=False, input_shape=(IMAGE_RESOLUTION, IMAGE_RESOLUTION, CHANNELS))

base_model.trainable = True
print('count layers of vgg16 : ' + str(len(base_model.layers)))

base_model.summary()

# Freeze four convolution blocks
for layer in base_model.layers[:15]:
    layer.trainable = False

# Make sure you have frozen the correct layers
for i, layer in enumerate(base_model.layers):
    print(i, layer.name, layer.trainable)

print('count layers of ' + algorithm + ' : ' + str(len(base_model.layers)))

base_model.summary()

cnn_model = tf.keras.Sequential()
cnn_model.add(base_model)
cnn_model.add(layers.GlobalMaxPool2D())
cnn_model.add(layers.Dense(number_of_classes, activation='softmax'))

opt = tf.keras.optimizers.Adam(learning_rate=1e-2)

loss = tf.keras.losses.SparseCategoricalCrossentropy()

metrics = ['accuracy']

cnn_model.compile(
    optimizer=opt,
    loss=loss,
    metrics=metrics
)

cnn_model.summary()

learning_reduce = ReduceLROnPlateau(monitor='val_accuracy', factor=0.6, patience=5, verbose=1, mode='max', min_lr=5e-7)

early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)

EPOCHS = 30

history = cnn_model.fit(train_ds, epochs=EPOCHS, validation_data=test_ds, callbacks=[learning_reduce,early_stopping])

print("Evaluate on train data")
results = cnn_model.evaluate(train_ds, batch_size=32)
print("train loss, train acc:", results)

print("Evaluate on test data")
results = cnn_model.evaluate(test_ds, batch_size=32)
print("test loss, test acc:", results)

process_dataset_with_model_ds(train_ds_original, model=cnn_model)
process_dataset_with_model_ds(test_ds_original, model=cnn_model)
display_training_history(history, EPOCHS)

In [ ]:
def preprocess(item):
    # print(item)
    image = item['image']
    # image = image // 255.0
    label = item['label']
    return image, label

def preprocess_ds(item):
    # print(item)
    image = item[0]
    # image = image // 255.0
    label = item[1]
    return image, label

def preprocess_dataset(dataset_train, columns : int = 5, rows : int = 5):
    item_list = []
    counter = 0
    for foo in dataset_train:
        image, label = preprocess_ds(foo)
        if counter > columns*rows+1:
          break
        counter += 1
        label_name = info.features['label'].names[label]
        item_dict = {"label": label, "image": image, "label_name": label_name}
        item_list.append(item_dict)

    fig = plt.figure(figsize=(16, 16))
    for i in range(1, columns*rows +1):
        item = item_list[i]
        label = item["label"]
        image = item['image']
        label_name = item["label_name"]

        image_resize = tf.image.resize(image, (224, 224)) / 255.0
        image_redim = tf.expand_dims(image_resize, axis=0) #.shape.as_list()
        prediction = model.predict(image_redim)
        b = tf.math.argmax(prediction[0])

        max_item = max(prediction[0])

        fig.add_subplot(rows, columns, i)
        plt.imshow(image.numpy()[: ,: , 0].astype(np.float32), cmap = plt.get_cmap("plasma"))
        plt.title(label=label_name)
    plt.show()

def process_dataset_with_model_ds(dataset_train, model, height : int = 32, width : int = 32, columns : int = 5, rows : int = 5):
    item_list = []
    counter = 0
    for foo in dataset_train:
        image, label = preprocess_ds(foo)
        # print(foo[0])
        # print(foo[1])
        if counter > columns*rows+1:
          break
        counter += 1
        label_name = info.features['label'].names[label]
        item_dict = {"label": label, "image": image, "label_name": label_name}
        item_list.append(item_dict)

    fig = plt.figure(figsize=(16, 16))
    for i in range(1, columns*rows +1):
        item = item_list[i]
        label = item["label"]
        image = item['image']
        label_name = item["label_name"]

        image_resize = tf.image.resize(image, (224, 224)) / 255.0
        image_redim = tf.expand_dims(image_resize, axis=0) #.shape.as_list()
        prediction = model.predict(image_redim)
        b = tf.math.argmax(prediction[0])

        max_item = max(prediction[0])
        index_value = [index for index, item in enumerate(prediction[0]) if item == max_item]
        index_value_int = int(index_value[0])

        label_name = info.features['label'].names[label]
        prediction_label_name = info.features['label'].names[index_value_int]

        color = "blue" if label == index_value_int else "red"

        label_finished = label_name + ' ' + str(label.numpy()) + ' / ' + prediction_label_name + ' ' + str(index_value_int) + ' '

        fig.add_subplot(rows, columns, i)
        plt.imshow(image.numpy()[: ,: , 0].astype(np.float32), cmap = plt.get_cmap("plasma"))
        plt.title(label=label_finished, color=color)
    plt.show()

def process_dataset_with_model(dataset_train, model, height : int = 32, width : int = 32, columns : int = 5, rows : int = 5):
    item_list = []
    counter = 0
    for foo in dataset_train:
        image, label = preprocess(foo)
        if counter > columns*rows+1:
          break
        counter += 1
        label_name = info.features['label'].names[label]
        item_dict = {"label": label, "image": image, "label_name": label_name}
        item_list.append(item_dict)

    fig = plt.figure(figsize=(16, 16))
    for i in range(1, columns*rows +1):
        item = item_list[i]
        label = item["label"]
        image = item['image']
        label_name = item["label_name"]

        image_resize = tf.image.resize(image, (224, 224)) / 255.0
        image_redim = tf.expand_dims(image_resize, axis=0) #.shape.as_list()
        prediction = model.predict(image_redim)
        b = tf.math.argmax(prediction[0])

        max_item = max(prediction[0])
        index_value = [index for index, item in enumerate(prediction[0]) if item == max_item]
        index_value_int = int(index_value[0])

        label_name = info.features['label'].names[label]
        prediction_label_name = info.features['label'].names[index_value_int]

        color = "blue" if label == index_value_int else "red"

        label_finished = label_name + ' ' + str(label.numpy()) + ' / ' + prediction_label_name + ' ' + str(index_value_int) + ' '

        fig.add_subplot(rows, columns, i)
        plt.imshow(image.numpy()[: ,: , 0].astype(np.float32), cmap = plt.get_cmap("plasma"))
        plt.title(label=label_finished, color=color)
    plt.show()

def format_image(image, label, image_res = IMAGE_RESOLUTION):
    image = tf.image.resize(image, (image_res, image_res))//255.0
    return image, label

In [ ]:
dataset_name = 'tf_flowers'

dataset, info = tfds.load(name=dataset_name, with_info=True)
dataset_train = dataset['train']

preprocess_dataset(dataset_train)

(train_ds, test_ds), dataset_info = tfds.load(dataset_name,
                                              with_info=True,
                                              as_supervised=True,
                                              split=["train[:80%]", "train[:20%]"])

train_ds_original = train_ds
test_ds_original = test_ds

# Preprocess the dataset
# def preprocess_data(data):
#     image = data['image']
#     label = data['label']
#     # Resize the image to your desired size
#     image = tf.image.resize(image, (64, 64))
#     # Normalize the image
#     image = image / 255.0
#     return image, label

# # Apply preprocessing to each image
# dataset = dataset.map(preprocess_data)

# data_generator = ImageDataGenerator(
#     rotation_range=10,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     horizontal_flip=True,
#     vertical_flip=False
# )

# train_x_batches = dataset.map(lambda x, y: x)
# train_y_batches = dataset.map(lambda x, y: y)

# train_x_list = tf.constant(list(train_x_batches.as_numpy_iterator()))
# train_y_list = tf.constant(list(train_y_batches.as_numpy_iterator()))

# print(train_x_list[:5])

# print(train_y_list[:5])

# train_generator = data_generator.flow(train_x_list, train_y_list)

number_of_classes = dataset_info.features['label'].num_classes
print(dataset_info)
num_training_examples = 0
num_validation_examples = 0

for _ in train_ds:
    num_training_examples +=1

for _ in test_ds:
    num_validation_examples +=1

BATCH_SIZE = 32
train_ds = train_ds.shuffle(num_training_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
test_ds = test_ds.map(format_image).batch(BATCH_SIZE).prefetch(1)

In [ ]:
print('Total Number of Classes: {}'.format(number_of_classes))
print('Size of images: {}'.format(IMAGE_RESOLUTION))
print('Total Number of Training Images: {}'.format(num_training_examples))
print('Total Number of Validation Images: {} \n'.format(num_validation_examples))

In [ ]:
# cnn_model = Sequential()

# # Block 1
# cnn_model.add(layers.Conv2D(64, (3, 3), activation=activation_function, padding='same', input_shape=(IMAGE_RESOLUTION, IMAGE_RESOLUTION, 3)))
# cnn_model.add(layers.Conv2D(64, (3, 3), activation=activation_function, padding='same'))
# cnn_model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

# # Block 2
# cnn_model.add(layers.Conv2D(128, (3, 3), activation=activation_function, padding='same'))
# cnn_model.add(layers.Conv2D(128, (3, 3), activation=activation_function, padding='same'))
# cnn_model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

# # Block 3
# cnn_model.add(layers.Conv2D(256, (3, 3), activation=activation_function, padding='same'))
# cnn_model.add(layers.Conv2D(256, (3, 3), activation=activation_function, padding='same'))
# cnn_model.add(layers.Conv2D(256, (3, 3), activation=activation_function, padding='same'))
# cnn_model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

# # Block 4
# cnn_model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
# cnn_model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
# cnn_model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
# cnn_model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

# # Block 5
# cnn_model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
# cnn_model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
# cnn_model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
# cnn_model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

# # Classification layers
# cnn_model.add(layers.Flatten())
# cnn_model.add(layers.Dense(4096, activation='relu'))
# cnn_model.add(layers.Dense(4096, activation='relu'))
# cnn_model.add(layers.Dense(5, activation='softmax'))

FILTER1_SIZE = 32
FILTER2_SIZE = 64
FILTER_SHAPE = (3, 3)
FULLY_CONNECT_NUM = 128
POOL_SHAPE = (2, 2)
BATCH_SIZE = 32

activation_function = 'LeakyReLU'

cnn_model = tf.keras.Sequential([
    layers.Conv2D(FILTER1_SIZE, FILTER_SHAPE, input_shape=(IMAGE_RESOLUTION, IMAGE_RESOLUTION, 3)),
    layers.MaxPooling2D(POOL_SHAPE),
    layers.Conv2D(FILTER2_SIZE, FILTER_SHAPE, activation=activation_function),
    layers.MaxPooling2D(POOL_SHAPE),
    layers.Conv2D(FILTER2_SIZE, FILTER_SHAPE, activation=activation_function),
    layers.MaxPooling2D(POOL_SHAPE),
    layers.Flatten(),
    layers.Dense(FULLY_CONNECT_NUM, activation=activation_function),
    layers.Dense(number_of_classes, activation='softmax')
])

# Preprocess the dataset
def preprocess_data(data):
    image = data['image']
    label = data['label']
    # Resize the image to your desired size
    image = tf.image.resize(image, (IMAGE_RESOLUTION, IMAGE_RESOLUTION))
    # Normalize the image
    image = image / 255.0
    return image, label

# Apply preprocessing to each image
# train_ds = train_ds.map(preprocess_data)
# test_ds = test_ds.map(preprocess_data)

opt = tf.keras.optimizers.Adam(learning_rate=0.001)
# opt = tf.keras.optimizers.AdamW(learning_rate=0.01)
# opt = tf.keras.optimizers.RMSprop()

loss = tf.keras.losses.SparseCategoricalCrossentropy()

metrics = ['accuracy']

cnn_model.compile(
    optimizer=opt,
    loss=loss,
    metrics=metrics
)

cnn_model.summary()

learning_reduce = ReduceLROnPlateau(monitor='val_accuracy', factor=0.6, patience=5, verbose=1, mode='max', min_lr=5e-7)

# early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)

EPOCHS = 30

# history = cnn_model.fit(train_generator, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=test_ds) #train_ds

history = cnn_model.fit(train_ds, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=test_ds, callbacks=[learning_reduce]) #train_ds

print("Evaluate on train data")
results = cnn_model.evaluate(train_ds, batch_size=32)
print("train loss, train acc:", results)

print("Evaluate on test data")
results = cnn_model.evaluate(test_ds, batch_size=32)
print("test loss, test acc:", results)

process_dataset_with_model(dataset_train, cnn_model, height=224, width=224)



In [ ]:
# dataset_train = dataset_train.map(preprocess_data)
#process_dataset_with_model(dataset_train, cnn_model, height=224, width=224)
print(dataset_train)
process_dataset_with_model_ds(train_ds_original, model=cnn_model)
process_dataset_with_model_ds(test_ds_original, model=cnn_model)

# print(train_ds.element_spec)
# print(train_ds.element_spec[0],train_ds.element_spec[1])
# for s in train_ds.element_spec:
#   print(s.name)
# my_information = dict({'image': train_ds.element_spec[0] ,'label': train_ds.element_spec[1]})
# print(my_information)
# process_dataset_with_model(my_information, cnn_model, height=224, width=224)
# process_dataset_with_model(test_ds, cnn_model, height=224, width=224)

### using data augmentation

In [ ]:
dataset_name = 'tf_flowers'

dataset, info = tfds.load(name=dataset_name, with_info=True)
dataset_train = dataset['train']

#preprocess_dataset(dataset_train)

(train_ds, test_ds), dataset_info = tfds.load(dataset_name,
                                              with_info=True,
                                              as_supervised=True,
                                              split=["train[:80%]", "train[:20%]"])
train_ds_original = train_ds
test_ds_original = test_ds

number_of_classes = dataset_info.features['label'].num_classes
# print(dataset_info)
num_training_examples = 0
num_validation_examples = 0

# BATCH_SIZE = 32
# train_ds = train_ds.shuffle(num_training_examples//4).batch(BATCH_SIZE).prefetch(1)
# test_ds = test_ds.batch(BATCH_SIZE).prefetch(1)

# print(f'len of train_ds {len(train_ds)}')
# print(f'type of train_ds {type(train_ds)}')

# print(f'dataset {dataset}')
# print(f'len of dataset {len(dataset)}')
# print(f'type of dataset {type(dataset)}')

# type of train_ds <class 'tensorflow.python.data.ops.prefetch_op._PrefetchDataset'>
# dataset {'train': <_PrefetchDataset element_spec={'image': TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}>}

# train_data = [(example.numpy(), label) for example, label in dataset_train]

# print(f'len of dataset {len(train_data)}')
# print(f'type of dataset {type(train_data)}')

# Preprocess the dataset
# def preprocess_data(data):
#     print(data)
#     image = data['image']
#     label = data['label']
#     # Resize the image to your desired size
#     image = tf.image.resize(image, (IMAGE_RESOLUTION, IMAGE_RESOLUTION))
#     # Normalize the image
#     image = image / 255.0
#     return image, label

def preprocess_data_ds(image_data, label_data):
    # print(image_data)
    # print(label_data)
    image = image_data
    label = label_data
    # Resize the image to your desired size
    image = tf.image.resize(image, (IMAGE_RESOLUTION, IMAGE_RESOLUTION))
    # Normalize the image
    image = image / 255.0
    return image, label

# Apply preprocessing to each image
# print(type(dataset_train))
# print(type(train_ds))
# dataset_train = dataset_train.map(preprocess_data)
train_ds = train_ds.map(preprocess_data_ds)
test_ds = test_ds.map(preprocess_data_ds)
# print(dataset_train)
# print(train_ds)

data_generator = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False,
    validation_split = 0.2
)

train_x_batches = train_ds.map(lambda x, y: x)
train_y_batches = train_ds.map(lambda x, y: y)

test_x_batches = test_ds.map(lambda x, y: x)
test_y_batches = test_ds.map(lambda x, y: y)

train_x_list = tf.constant(list(train_x_batches.as_numpy_iterator()))
train_y_list = tf.constant(list(train_y_batches.as_numpy_iterator()))

test_x_list = tf.constant(list(test_x_batches.as_numpy_iterator()))
test_y_list = tf.constant(list(test_y_batches.as_numpy_iterator()))

train_generator = data_generator.flow(train_x_list, train_y_list)

# print(train_generator)
# print(type(train_generator))
# print(len(train_generator))

FILTER1_SIZE = 64
FILTER2_SIZE = 128
FILTER_SHAPE = (2, 2)
FULLY_CONNECT_NUM = 156
POOL_SHAPE = (2, 2)
BATCH_SIZE = 32

activation_function = 'LeakyReLU'

cnn_model = tf.keras.Sequential([
    layers.Conv2D(FILTER1_SIZE, FILTER_SHAPE, input_shape=(IMAGE_RESOLUTION, IMAGE_RESOLUTION, 3)),
    layers.MaxPooling2D(POOL_SHAPE),
    layers.Conv2D(FILTER2_SIZE, FILTER_SHAPE, activation=activation_function),
    layers.MaxPooling2D(POOL_SHAPE),
    layers.Conv2D(FILTER2_SIZE, FILTER_SHAPE, activation=activation_function),
    layers.MaxPooling2D(POOL_SHAPE),
    layers.Flatten(),
    layers.Dense(FULLY_CONNECT_NUM, activation=activation_function),
    layers.Dense(number_of_classes, activation='softmax')
])

# opt = tf.keras.optimizers.Adam(learning_rate=0.001)
opt = tf.keras.optimizers.AdamW(learning_rate=0.001)

loss = tf.keras.losses.SparseCategoricalCrossentropy()

metrics = ['accuracy']

cnn_model.compile(
    optimizer=opt,
    loss=loss,
    metrics=metrics
)

cnn_model.summary()

learning_reduce = ReduceLROnPlateau(monitor='val_accuracy', factor=0.6, patience=5, verbose=1, mode='max', min_lr=5e-7)

early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)

EPOCHS = 50

history = cnn_model.fit(train_generator, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(test_x_list, test_y_list), callbacks=[learning_reduce, early_stopping]) #train_ds ,

# print("Evaluate on train data")
# results = cnn_model.evaluate(dataset_train, batch_size=32)
# print("train loss, train acc:", results)

# print("Evaluate on train data")
# results = cnn_model.evaluate(train_ds_original, batch_size=32)
# print("train loss, train acc:", results)

# print("Evaluate on test data")
# results = cnn_model.evaluate(test_ds_original, batch_size=32)
# print("test loss, test acc:", results)

process_dataset_with_model(dataset_train, cnn_model, height=224, width=224)
process_dataset_with_model_ds(train_ds_original, cnn_model, height=224, width=224)
process_dataset_with_model_ds(test_ds_original, cnn_model, height=224, width=224)


In [ ]:
print('show train data')
process_dataset_with_model_ds(train_ds_original, model=cnn_model)
print('show test data')
process_dataset_with_model_ds(test_ds_original, model=cnn_model)
print('show train generator data')
results = cnn_model.evaluate(train_generator, batch_size=32)

In [ ]:
dataset_name = 'tf_flowers'

dataset, info = tfds.load(name=dataset_name, with_info=True)
dataset_train = dataset['train']

preprocess_dataset(dataset_train)

(train_ds, test_ds), dataset_info = tfds.load(dataset_name,
                                              with_info=True,
                                              as_supervised=True,
                                              split=["train[:80%]", "train[:20%]"])

# print(f'len of train_ds {len(train_ds)}')
# print(f'type of train_ds {type(train_ds)}')

# print(f'dataset {dataset}')
# print(f'len of dataset {len(dataset)}')
# print(f'type of dataset {type(dataset)}')

# type of train_ds <class 'tensorflow.python.data.ops.prefetch_op._PrefetchDataset'>
# dataset {'train': <_PrefetchDataset element_spec={'image': TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}>}

# train_data = [(example.numpy(), label) for example, label in dataset_train]

# print(f'len of dataset {len(train_data)}')
# print(f'type of dataset {type(train_data)}')

# Preprocess the dataset
def preprocess_data(data):
    print(data)
    image = data['image']
    label = data['label']
    # Resize the image to your desired size
    image = tf.image.resize(image, (IMAGE_RESOLUTION, IMAGE_RESOLUTION))
    # Normalize the image
    image = image / 255.0
    return image, label

# Apply preprocessing to each image
print(type(dataset_train))
print(type(train_ds.element_spec))
dataset_train = dataset_train.map(preprocess_data)
train_ds = train_ds.element_spec.map(preprocess_data)

# data_generator = ImageDataGenerator(
#     rotation_range=10,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     horizontal_flip=True,
#     vertical_flip=False
# )

# train_x_batches = dataset.map(lambda x, y: x)
# train_y_batches = dataset.map(lambda x, y: y)

# train_x_list = tf.constant(list(train_x_batches.as_numpy_iterator()))
# train_y_list = tf.constant(list(train_y_batches.as_numpy_iterator()))

# print(train_x_list[:5])

# print(train_y_list[:5])

# train_generator = data_generator.flow(train_x_list, train_y_list)

number_of_classes = dataset_info.features['label'].num_classes
# print(dataset_info)
num_training_examples = 0
num_validation_examples = 0

for _ in train_ds:
    num_training_examples +=1

for _ in test_ds:
    num_validation_examples +=1

BATCH_SIZE = 32
train_ds = train_ds.shuffle(num_training_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
test_ds = test_ds.map(format_image).batch(BATCH_SIZE).prefetch(1)

FILTER1_SIZE = 64
FILTER2_SIZE = 128
FILTER_SHAPE = (2, 2)
FULLY_CONNECT_NUM = 156
POOL_SHAPE = (2, 2)
BATCH_SIZE = 32

activation_function = 'LeakyReLU'

cnn_model = tf.keras.Sequential([
    layers.Conv2D(FILTER1_SIZE, FILTER_SHAPE, input_shape=(IMAGE_RESOLUTION, IMAGE_RESOLUTION, 3)),
    layers.MaxPooling2D(POOL_SHAPE),
    layers.Conv2D(FILTER2_SIZE, FILTER_SHAPE, activation=activation_function),
    layers.MaxPooling2D(POOL_SHAPE),
    layers.Conv2D(FILTER2_SIZE, FILTER_SHAPE, activation=activation_function),
    layers.MaxPooling2D(POOL_SHAPE),
    layers.Flatten(),
    layers.Dense(FULLY_CONNECT_NUM, activation=activation_function),
    layers.Dense(number_of_classes, activation='softmax')
])

# opt = tf.keras.optimizers.Adam(learning_rate=0.0005)
opt = tf.keras.optimizers.AdamW(learning_rate=0.001)

loss = tf.keras.losses.SparseCategoricalCrossentropy()

metrics = ['accuracy']

cnn_model.compile(
    optimizer=opt,
    loss=loss,
    metrics=metrics
)

cnn_model.summary()

learning_reduce = ReduceLROnPlateau(monitor='val_accuracy', factor=0.6, patience=5, verbose=1, mode='max', min_lr=5e-7)

early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)

EPOCHS = 1

# history = cnn_model.fit(train_generator, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=test_ds) #train_ds

history = cnn_model.fit(train_ds, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=test_ds, callbacks=[learning_reduce, early_stopping]) #train_ds

print("Evaluate on train data")
results = cnn_model.evaluate(train_ds, batch_size=32)
print("train loss, train acc:", results)

print("Evaluate on test data")
results = cnn_model.evaluate(test_ds, batch_size=32)
print("test loss, test acc:", results)

process_dataset_with_model(dataset_train, cnn_model, height=224, width=224)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(16, 10))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()